In [155]:
import pandas as pd
from PIL import Image
from matplotlib import pyplot as plt
from torchvision.utils import draw_bounding_boxes
from torchvision import transforms
import torch
import os

In [69]:
img_path = 'bbox_test/API_LatLng.jpg'
ann_path = 'bbox_test/144308_API_LatLng.txt'

In [70]:
convert_tensor = transforms.ToTensor()

img = Image.open(img_path).convert('RGB')
img_tensor = convert_tensor(img)

img_int = torch.tensor(img_tensor*255, dtype=torch.uint8)


C:\Users\as279\AppData\Local\Temp/ipykernel_20080/1623778424.py:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  img_int = torch.tensor(img_tensor*255, dtype=torch.uint8)


In [5]:
box = []

with open(ann_path) as f:
    for line in f.readlines():
       box.append(line) 

In [6]:
box

['1.000 0.133 0.546 0.113 0.136\n',
 '1.000 0.309 0.682 0.492 0.077\n',
 '2.000 0.602 0.692 0.078 0.069\n']

In [77]:
m = box[0]
m = list(filter(("\n").__ne__, m))
m = "".join(m)
m = m.split(' ')

In [78]:
m

['1.000', '0.133', '0.546', '0.113', '0.136']

In [49]:
box_df = pd.DataFrame(columns=["image_id", "type","xmin","ymin","xmax","ymax"])




In [50]:
box_df

,type,xmin,ymin,xmax,ymax
0,1.0,0.133,0.546,0.113,0.136


In [189]:
def getBoxInf(img_id, line):
    return [img_id] + toBbox_loc(toBbox_coord(line))

def toBbox_coord(fline):

    fline = list(filter(("\n").__ne__, fline))
    fline = "".join(fline)
    fline = fline.split(' ')
    box_coord = [float(fline[i]) for i in range(len(fline))]
    return box_coord

def toBbox_loc(box_coord):
    loc_box = [box_coord[0], 640*(box_coord[1]-box_coord[3]/2), 640*(box_coord[2]-box_coord[4]/2), 640*(box_coord[1]+box_coord[3]/2), 640*(box_coord[2]+box_coord[4]/2)]
    return loc_box

In [199]:
def getBbox(img_id):

    '''
    Given: image ID (ex. 144308_API_LatLng.txt)
    Read each line in the txt file, which are the information of bounding box in a image, and add them into the list 'box'
    box: type, x, y, width, height in string type
    by toBbox_coord: translate it into list type: [type, x, y, width, height]
    by toBbox_loc  : translate it into list type for model using: [type, xmin, ymin, xmax, ymax]
    finally, return a list made up of [image id, type, xmin, ymin, xmax, ymax]
    '''

    path = 'bbox_test/' + str(img_id)
    box = []
    with open(path) as f:
        for line in f.readlines():
            box.append(line) 
    df = {"image_id": [], "type": [], "xmin": [], "ymin": [], "xmax": [], "ymax": []}
    for line in box:
        df["image_id"].append(img_id)
        df["type"].append(str(getBoxInf(img_id, line)[1]))
        df["xmin"].append(float(getBoxInf(img_id, line)[2]))
        df["ymin"].append(float(getBoxInf(img_id, line)[3]))
        df["xmax"].append(float(getBoxInf(img_id, line)[4]))
        df["ymax"].append(float(getBoxInf(img_id, line)[5]))
    bbox = []
    for line in box:
        bbox.append([float(getBoxInf(img_id, line)[2]), float(getBoxInf(img_id, line)[3]), float(getBoxInf(img_id, line)[4]), float(getBoxInf(img_id, line)[5])])
    return df, bbox  





In [220]:
getBbox('144308_API_LatLng.txt')[0]['type']

['1.0', '1.0', '2.0']

In [192]:
df_ = pd.DataFrame(getBbox('144308_API_LatLng.txt')[0])

In [148]:
df_

,image_id,type,xmin,ymin,xmax,ymax
0,144308_API_LatLng.txt,1.0,48.96,305.92,121.28,392.96
1,144308_API_LatLng.txt,1.0,40.32,411.84,355.20,461.12
2,144308_API_LatLng.txt,2.0,360.32,420.80,410.24,464.96


In [150]:
bbox = [df_["xmin"][0], df_["ymin"][0], df_["xmax"][0], df_["ymax"][0]]

In [ ]:
fig = plt.figure(figsize=(14, 10))
plt.imshow(draw_bounding_boxes(img_int, torch.Tensor([[df_["xmin"][i], df_["ymin"][i], df_["xmax"][i], df_["ymax"][i]] for i in range(3)]),  colors='red', width=2).permute(1, 2, 0))

In [ ]:
"/home/tonio1314/CVDL/DataSet/DataSet_C/100013/API_Address.jpg"

"/home/tonio1314/CVDL/kuokuo/object_detection/annotation/b_box/100007_API_LatLng.txt"

In [230]:
datas_dic =  {"index": [], "img_id": [], "bbox": [], "bbox_num": [], "img_root": [], "anno_root": [], "label": [], "split": []}

for path in os.listdir('bbox_test/test/anno'):
    datas_dic['anno_root'].append('bbox_test/test/anno/'+path)
    datas_dic['split'].append("train")
    print(path)

for i in range(len(datas_dic['anno_root'])):
    datas_dic['img_id'].append(datas_dic['anno_root'][i].split('/')[3])
    datas_dic['index'].append(i)
    
for i in range(len(datas_dic['anno_root'])):
    datas_dic['bbox'].append(getBbox(datas_dic['img_id'][i])[1])
    datas_dic['bbox_num'].append(len(getBbox(datas_dic['img_id'][i])[1]))
    datas_dic['label'].append(getBbox(datas_dic['img_id'][i])[0]['type'])
    datas_dic["img_root"].append('bbox_test/test/dataset/'+datas_dic['img_id'][0].split('_')[0]+'/'+(datas_dic["img_id"][0].split('_')[1] + "_" + datas_dic["img_id"][0].split('_')[2]).split('.')[0] + ".jpg")

144308_API_LatLng.txt


In [231]:
datas_dic

{'index': [0],
 'img_id': ['144308_API_LatLng.txt'],
 'bbox': [[[48.96000000000001, 305.92, 121.28, 392.96000000000004],
   [40.32, 411.84000000000003, 355.19999999999993, 461.12],
   [360.31999999999994, 420.79999999999995, 410.24, 464.9599999999999]]],
 'bbox_num': [3],
 'img_root': ['bbox_test/test/dataset/144308/API_LatLng.jpg'],
 'anno_root': ['bbox_test/test/anno/144308_API_LatLng.txt'],
 'label': [['1.0', '1.0', '2.0']],
 'split': ['train']}

In [232]:
datas_df = pd.DataFrame(datas_dic)

In [233]:
datas_df

,index,img_id,bbox,bbox_num,img_root,anno_root,label,split
0,0,144308_API_LatLng.txt,"[[48.96000000000001, 305.92, 121.28, 392.96000...",3,bbox_test/test/dataset/144308/API_LatLng.jpg,bbox_test/test/anno/144308_API_LatLng.txt,"[1.0, 1.0, 2.0]",train
